In [ ]:
%run query.ipynb

In [3]:
from pyrogram import Client, filters
from pyrogram.raw.functions.chatlists import get_chatlist_updates
from pyrogram.raw.functions.channels.get_channels import GetChannels
from pyrogram.enums import ChatType
from dotenv import load_dotenv
import random
import time
import os
load_dotenv()

TELEGRAM_API_ID = os.environ["TELEGRAM_API_ID"]
TELEGRAM_API_HASH = os.environ["TELEGRAM_API_HASH"]
TELEGRAM_PHONE = os.environ["TELEGRAM_PHONE"]
TELEGRAM_PASS = os.environ["TELEGRAM_PASS"]
app = Client("listener2", api_id=TELEGRAM_API_ID, api_hash=TELEGRAM_API_HASH, phone_number=TELEGRAM_PHONE, password=TELEGRAM_PASS)
client = await app.start()

In [4]:
from pyrogram.types import Dialog
async def get_channel_discussion_group(d: Dialog):
    channel_name, group_name, join_result = d.chat.title, None, None
    channel_id = d.chat.id
    message_id = d.top_message.id
    try: discussion_message = await app.get_discussion_message(channel_id, message_id)
    except: discussion_message = None
    discussion_group = discussion_message.chat if discussion_message else None
    if discussion_group:
        group_name = discussion_group.title
        group_id = discussion_group.id
        return group_id, group_name
    return None, None

In [5]:
from pyrogram.types import Dialog
async def join_channel_discussion_group(d: Dialog):
    channel_name = d.chat.title
    group_id, group_name = get_channel_discussion_group(d)
    join_result = None
    if group_id is not None:
        try: join_result = await app.join_chat(group_id)
        except: join_result = "SENT"
        if join_result != None and join_result != "SENT":
            join_result = "SUCCESS"
    return channel_name, group_name, join_result

In [6]:
it = client.get_dialogs()
l = []
async for row in it:
    l.append(row)
else:
    print("REACHED ELSE")
print(len(l))

REACHED ELSE
151


In [28]:
channels = [x for x in l if x.chat.type == ChatType.CHANNEL]
for dialog in channels:
    channel_name, group_name, result = await join_channel_discussion_group(dialog)
    print(f"""Result for "{channel_name}": group_name is "{group_name}" status = {result}""")

Result for "REDACTED" status = None
Result for "REDACTED" status = SUCCESS
Result for "REDACTED" status = SUCCESS
Result for "REDACTED" status = None
Result for "REDACTED" status = SENT
Result for "REDACTED" status = None
Result for "REDACTED" status = SUCCESS


[listener2] Waiting for 3 seconds before continuing (required by "channels.JoinChannel")


Result for "REDACTED" status = SENT


[listener2] Waiting for 4 seconds before continuing (required by "channels.JoinChannel")


Result for "REDACTED" status = SENT


[listener2] Waiting for 4 seconds before continuing (required by "channels.JoinChannel")


Result for "REDACTED" status = SUCCESS


[listener2] Waiting for 4 seconds before continuing (required by "channels.JoinChannel")


Result for "REDACTED" status = SUCCESS
Result for "REDACTED" status = SUCCESS
Result for "REDACTED" status = SENT
Result for "REDACTED" status = None
Result for "REDACTED" status = SENT
Result for "REDACTED" status = None
Result for "REDACTED" status = None
Result for "REDACTED" status = SENT
Result for "REDACTED" status = None
Result for "REDACTED" status = SENT
Result for "REDACTED" status = SENT
Result for "REDACTED" status = SUCCESS
Result for "REDACTED" status = SENT
Result for "REDACTED" status = SENT
Result for "REDACTED" status = SENT
Result for "REDACTED" status = None
Result for "REDACTED" status = SENT
Result for "REDACTED" status = SENT
Result for "REDACTED" status = SENT
Result for "REDACTED" status = SENT
Result for "REDACTED" status = SENT
Result for "REDACTED" status = SENT
Result for "REDACTED" status = SENT
Result for "REDACTED" status = None
Result for "REDACTED" status = None
Result for "REDACTED" status = None
Result for "REDACTED" status = None
Result for "REDACTE

In [7]:
channels = [x for x in l if x.chat.type == ChatType.CHANNEL]
channel_group = dict()
group_channel = dict()
for dialog in channels:
    group_id, group_name = await get_channel_discussion_group(dialog)
    channel_group[dialog.chat.id] = group_id
    if group_id is not None:
        group_channel[group_id] = dialog.chat.id

In [42]:
"""
CREATE TABLE IF NOT EXISTS chats (
    id BIGINT NOT NULL,
    type TINYINT NOT NULL,
    first_name TEXT CHARACTER SET utf8mb4,
    last_name TEXT CHARACTER SET utf8mb4,
    username TEXT,
    is_verified BOOLEAN,
    is_scam BOOLEAN,
    is_restricted BOOLEAN,
    is_bot BOOLEAN,
    is_support BOOLEAN,
    members_count INTEGER,
    group_id BIGINT,
    channel_id BIGINT,
    PRIMARY KEY (id)
);
"""

# add group_id, channel_id to chats table
# alter table chats add column group_id BIGINT;
# x1=list(query("ALTER TABLE chats ADD COLUMN group_id BIGINT;"))
# x2=list(query("ALTER TABLE chats ADD COLUMN channel_id BIGINT;"))
# list(query("ALTER TABLE chats ADD COLUMN title TEXT CHARACTER SET utf8mb4"))

In [8]:
chat_cols = (list(get_cols("chats")))

In [10]:
chat_type_dict = {
    ChatType.CHANNEL: 1,
    ChatType.GROUP: 2,
    ChatType.SUPERGROUP: 3,
    ChatType.PRIVATE: 4,
    ChatType.BOT: 5,
}

from pyrogram.types import Chat, Dialog
from typing import Dict, Optional
def normalize_chat(d: Dialog) -> Dict[str, Optional[str|int]]:
    chat: Chat = d.chat
    chat_dict = dict()
    chat_dict["id"] = chat.id
    chat_dict["type"] = chat_type_dict[chat.type]
    chat_dict["title"] = chat.title
    chat_dict["first_name"] = chat.first_name
    chat_dict["last_name"] = chat.last_name
    chat_dict["username"] = chat.username
    chat_dict["is_verified"] = chat.is_verified
    chat_dict["is_scam"] = chat.is_scam or chat.is_fake
    chat_dict["is_restricted"] = chat.is_restricted
    chat_dict["is_bot"] = getattr(chat, "is_bot", False)
    chat_dict["is_support"] = chat.is_support
    chat_dict["members_count"] = chat.members_count
    chat_dict["channel_id"] = group_channel.get(chat.id, None)
    chat_dict["group_id"] = channel_group.get(chat.id, None)
    assert set(chat_dict.keys()) == set(chat_cols), f"{set(chat_dict.keys()) - set(chat_cols)} {set(chat_cols) - set(chat_dict.keys())}"
    return chat_dict

chat_dicts = []
for d in l:
    chat_dicts.append(normalize_chat(d))

In [ ]:
insert(chat_dicts, "messages")

In [11]:
list(query("SELECT COUNT(*) FROM chats"))

[(151,)]

In [30]:
app.stop()

<coroutine object Stop.stop at 0x7f4faf6b8040>